In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'label','msgs':'message','diffs':'diff'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_4451/2507598078.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,language,commit,commit_url,label,user,repo,feature,diff,message
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,1,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,1,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,1,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,0,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/eval.csv',index=False)

In [5]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=20, random_state=42))

/tmp/ipykernel_4451/240346103.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=20, random_state=42))


In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,language,commit,commit_url,label,user,repo,feature,diff,message
401,Go,ed09b4ea6a17566b546a9d3ae76876636710c6a0,https://github.com/v2ray/v2ray-core/commit/ed0...,0,v2ray,v2ray-core,"[3, 49, 2, 8, 0, 0, 0, 2, 29, 0, 2, 0, 0, 0, 0...",diff --git a/common/net/json/host.go b/common/...,support domain as host in VMess outbound confi...
907,Rust,e67010b5e2eaa5a5ddb756d7ddc2dd0194f09a0a,https://github.com/denoland/deno/commit/e67010...,0,denoland,deno,"[2, 25, 1, 7, 0, 0, 0, 2, 35, 0, 0, 0, 0, 0, 0...",diff --git a/cli/file_fetcher.rs b/cli/file_fe...,feat(cli/compile): Support data uri dynamic im...
1385,Java,74066765bb8ca85864f75bde6091f96207f988cf,,0,apache,hbase,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/main/java/org/apache/hadoop/h...,HBASE-2889 Tool to look at HLogs -- parse and-...
448,Go,8d9e4345207013c55a8719995ec4ad4e9be466f2,https://github.com/FiloSottile/mkcert/commit/8...,0,FiloSottile,mkcert,"[1, 9, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/truststore_nss.go b/truststore_ns...,truststore_nss: support multiple NSS databases...
653,JavaScript,145c46ff3630e55b6c50ff479f54661ef54124bd,https://github.com/webpack/webpack/commit/145c...,0,webpack,webpack,"[1, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/lib/dependencies/DelegatedExports...,feat(DelegatedExportsDependency): make seriali...
463,Go,73890f31af5f1247c1f5b8131f136d1045d6b5f6,https://github.com/minio/minio/commit/73890f31...,0,minio,minio,"[1, 18, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/.goreleaser.yml b/.goreleaser.yml...,add minisign verification for container builds...
1052,Rust,e6132809d27164077c2bca7f4cebdcea5e03eff0,https://github.com/dani-garcia/vaultwarden/com...,0,dani-garcia,vaultwarden,"[1, 34, 0, 10, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/src/api/core/organizations.rs b/s...,Implemented delete user from org and added che...
1442,Java,98a24828255cde13eaee893eafc9a049bb2183a7,,0,elastic,elasticsearch,"[0, 0, 0, 0, 0, 0, 0, 2, 38, 0, 0, 27, 0, 0, 0...",diff --git a/src/test/java/org/elasticsearch/t...,Testing: Add test rule to repeat tests on bind...
1502,Java,3e714ddd9f062e534c1ee028ac8b2269a062004b,,0,JetBrains,intellij-community,"[7, 295, 156, 82, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0...",diff --git a/platform-resources/src/META-INF/X...,IDEADEV-11483 IE conditional comments support-...
1370,Java,f64233fb2d9bcb77e9249d3bc497fd9d110e6a9f,,0,ponsonio,RxJava,"[1, 18, 0, 0, 17, 0, 0, 1, 27, 0, 0, 0, 0, 0, ...",diff --git a/src/main/java/rx/Single.java b/sr...,Add Single.fromCallable()--


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 1.1504: 100%|██████████| 8/8 [00:03<00:00,  2.45batch/s]


=============================train========================


Epoch 2/10 Loss: 1.0144: 100%|██████████| 8/8 [00:02<00:00,  2.73batch/s]


=============================train========================


Epoch 3/10 Loss: 0.8530: 100%|██████████| 8/8 [00:02<00:00,  2.73batch/s]


...

=============================train========================


Epoch 9/10 Loss: 0.0309: 100%|██████████| 8/8 [00:02<00:00,  2.75batch/s]


=============================train========================


Epoch 10/10 Loss: 0.0172: 100%|██████████| 8/8 [00:02<00:00,  2.79batch/s]


=============================train========================
Validation Accuracy: 0.5612
Precision: 0.6229
Recall: 0.5612
F1-Score: 0.5739
              precision    recall  f1-score   support

           0     0.3667    0.6471    0.4681        34
           1     0.4533    0.5667    0.5037        60
           2     0.7549    0.5385    0.6286       143

    accuracy                         0.5612       237
   macro avg     0.5250    0.5841    0.5335       237
weighted avg     0.6229    0.5612    0.5739       237



In [9]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.6050
Precision: 0.6932
Recall: 0.6050
F1-Score: 0.6284
              precision    recall  f1-score   support

           0     0.3167    0.5938    0.4130        32
           1     0.4627    0.6078    0.5254        51
           2     0.8468    0.6065    0.7068       155

    accuracy                         0.6050       238
   macro avg     0.5421    0.6027    0.5484       238
weighted avg     0.6932    0.6050    0.6284       238



In [11]:
from sklearn import metrics

In [12]:
cm = metrics.confusion_matrix(test_labels, test_predictions)
cm

array([[19,  8,  5],
       [ 8, 31, 12],
       [33, 28, 94]])